In [1]:
SECTIONS = ["latarbelakang", "rumusanmasalah", "tujuanpenelitian", "rangkumanpenelitianterkait", "metodologipenelitian"]
DATA_OUTPUT = "data/penilaian-data/80/output-bert.csv"
DATA_REFERENCE = "data/final-data/{SECTIONS}.csv"

In [2]:
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
import csv

# Fungsi untuk menghitung skor ROUGE
def calculate_rouge(generated, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(generated, reference)
    return (
        scores['rouge1'].fmeasure,
        scores['rouge2'].fmeasure,
        scores['rougeL'].fmeasure
    )

# Baca file output
output_df = pd.read_csv(DATA_OUTPUT)

# Inisialisasi dictionary untuk menyimpan hasil
results = {}

# Proses setiap bagian
for section in SECTIONS:
    # Baca file referensi untuk bagian ini
    ref_df = pd.read_csv(DATA_REFERENCE.format(SECTIONS=section))
    
    # Buat dictionary dari data referensi
    ref_dict = dict(zip(ref_df['nama_dokumen'], ref_df['summary']))
    
    for _, row in output_df.iterrows():
        nama_dokumen = row['nama_dokumen']
        generated_text = row[section]
        reference_text = ref_dict.get(nama_dokumen, '')
        
        # Hitung skor ROUGE
        r1, r2, rl = calculate_rouge(generated_text, reference_text) if reference_text else (0, 0, 0)
        
        # Simpan hasil
        if nama_dokumen not in results:
            results[nama_dokumen] = {'R1': [], 'R2': [], 'RL': []}
        
        results[nama_dokumen]['R1'].append(r1)
        results[nama_dokumen]['R2'].append(r2)
        results[nama_dokumen]['RL'].append(rl)

# Hitung rata-rata skor untuk setiap dokumen
final_results = []
for nama_dokumen, scores in results.items():
    final_results.append({
        'nama_dokumen': nama_dokumen,
        'R1': np.mean(scores['R1']),
        'R2': np.mean(scores['R2']),
        'RL': np.mean(scores['RL'])
    })

# Simpan hasil ke file CSV
output_file = 'data/rouge/rouge.csv'
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['nama_dokumen', 'R1', 'R2', 'RL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in final_results:
        writer.writerow(row)

print(f"Hasil perhitungan ROUGE telah disimpan di {output_file}")

Hasil perhitungan ROUGE telah disimpan di data/rouge/rouge.csv


In [3]:
import pandas as pd

# Baca file CSV yang telah kita simpan sebelumnya
df = pd.read_csv('data/rouge/rouge.csv')

# Daftar dokumen yang ingin ditampilkan
dokumen_target = [
    'Robert_Kualifikasi.txt',
    'MetaMeysawati_KUALIFIKASI(99216026).txt',
    'Kualifikasi_Rama Dian Syah.txt'
]

# Filter DataFrame untuk hanya menampilkan dokumen yang diminta
df_filtered = df[df['nama_dokumen'].isin(dokumen_target)]

# Tampilkan hasil
print(df_filtered.to_string(index=False))

# Hitung dan tampilkan rata-rata metrik
average_metrics = df_filtered[['R1', 'R2', 'RL']].mean()
print("\nRata-rata metrik:")
print(average_metrics.to_string())

                           nama_dokumen       R1       R2       RL
         Kualifikasi_Rama Dian Syah.txt 0.608213 0.431673 0.501974
MetaMeysawati_KUALIFIKASI(99216026).txt 0.464918 0.355807 0.411149
                 Robert_Kualifikasi.txt 0.458237 0.303286 0.364057

Rata-rata metrik:
R1    0.510456
R2    0.363588
RL    0.425727
